# Рекомендация тарифов

## 1. Откройте и изучите файл

Загрузим библиотеки

In [1]:
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

Выгрузим данные из файла в датафрейм

In [2]:
df=pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [31]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

## 2. Разбейте данные на выборки

In [5]:
features=df.drop(['is_ultra'], axis=1)

In [6]:
target=df['is_ultra']

In [7]:
features_train,features_test,target_train, target_test=train_test_split(features,target,test_size=0.2,random_state=42)

In [8]:
features_train,features_val, target_train,target_val=train_test_split(features_train,target_train,test_size=0.25,random_state=42)

## 3. Исследуйте модели

###### Применим логистическую регрессию

In [9]:
model_LogReg=LogisticRegression(random_state=42)

In [10]:
model_LogReg.fit(features_train,target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
prediction_LogReg=model_LogReg.predict(features_val)

In [12]:
accuracy_score(prediction_LogReg, target_val)

0.7216174183514774

###### Исследуем случайны лес

In [13]:
gs_rfc=GridSearchCV(RandomForestClassifier(),param_grid={'n_estimators' : range(2,20), 'max_depth' : range(2,20)})

In [14]:
gs_rfc.fit(features_train,target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [15]:
gs_rfc.best_score_

0.808091286307054

In [16]:
gs_rfc.best_params_

{'max_depth': 8, 'n_estimators': 19}

In [17]:
model_RanForClas=RandomForestClassifier(n_estimators=13,max_depth=10, random_state=42)

In [18]:
model_RanForClas.fit(features_train,target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=13,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

###### Исследуем дерево решений

In [19]:
gs_dt=GridSearchCV(DecisionTreeClassifier(),param_grid={ 'max_depth' : range(2,20),'splitter' : ('best','random'),'criterion' : ('gini', 'entropy'),'min_samples_split': range(2,6),'min_samples_leaf': range(1,6)}, return_train_score=True)

In [20]:
gs_dt.fit(features_train,target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': range(2, 20),
                    

In [21]:
gs_dt.best_score_

0.7930497925311203

In [22]:
gs_dt.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'splitter': 'random'}

In [23]:
model_DecTreeClas=(DecisionTreeClassifier(criterion= 'entropy',
                                          max_depth= 9,
                                          min_samples_leaf= 2,
                                          min_samples_split= 4,
                                          splitter='random', 
                                          random_state=42))

In [24]:
model_DecTreeClas.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=9,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='random')

Как видим, случайный лес и дерево решений дают очень близкие значения accuracy на валидатационно выборке.  
В следующем шаге можно будет проверить обе модели на тестовой

## 4. Проверьте модель на тестовой выборке

In [25]:
model_DecTreeClas.score(features_test, target_test)

0.7931570762052877

In [26]:
model_RanForClas.score(features_test, target_test)

0.8149300155520995

По результатам проверки на тестовой выборке accuracy случайного леса оказалась выше.

## 5. Проверьте модели на адекватность

In [27]:
model_Dummy=DummyClassifier(strategy='most_frequent')

In [28]:
model_Dummy.fit(features_train,target_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [29]:
prediction_Dummy=model_Dummy.predict(features_test)

In [30]:
accuracy_score(prediction_Dummy, target_test)

0.7076205287713841

Как видим, что бы предпологать о том, что модель адекватна, нам нужно получить значения accuracy выше 0.707, которую дает самая примитивная модель.

### Вывод

Мы открыли файл с данными, разбили их на 3 группы (обучающая, валидационная, тестовая) в отношении 0.6/0.2/0.2.  
Использовали 3 модели : логическая регрессия, случайный лес, дерево решений. По результатам подбора параметров и сравнения на валидационной выборке значение accuracy дерева решений и случайного леса оказались близки. На тестовой же выборке accuracy случайного леса было больше, поэтому можно отдать приоретет этой модели, но стоит отметить, что и дерева решений тест показал точность выше целевого.